# **ENGG680 - Introduction to Digital Engineering**
## *Lab Assignment 4: ML Applications - Vehicle Motion State Classification*

## Preliminary: Certificate of Work


*We, the undersigned, certify that this is our own work, which has been done expressly for this course, either without the assistance of any other party or where appropriate we have acknowledged the work of others. Further, we have read and understood the section in the university calendar on plagiarism/cheating/other academic misconduct and we are aware of the implications thereof. We request that the total mark for this assignment be distributed as follows among group members:*

|          | First Name | Last Name | Signature (Full Name, Date) | Hours | Contribution % |
|----------|------------|-----------|-----------------------------|-------|----------------|
| Member 1: | First Name | Last Name | Signature | Hours | Contribution |
| Member 2: | First Name | Last Name | Signature | Hours | Contribution |
| Member 3: | First Name | Last Name | Signature | Hours | Contribution |
| Member 4: | First Name | Last Name | Signature | Hours | Contribution |


---

## Section 1 - Motion Classification Using *All Features* Derived From Inertial Sensor Data (33 Marks)

---

### 1.1 Assignment datasets import (6 Marks)

---

In [1]:
# some library imports
import pandas as pd

First load the following datasets into dataframes.
* *UrbanNav-HK_Whampoa-20210521_sensors.csv* to dataframe variable named *cv_set*. This dataset is to be used for model training and hyper parameter tuning.
* *UrbanNav-HK_TST-20210517_sensors.csv* to dataframe variable named *test_set1*. This is first independent test dataset.
* *2020-03-14-16-45-35.csv* to dataframe variable named *test_set2*. This is the second independent test dataset.

In [2]:
# (2 marks)
# TODO: Load cross validation/training dataset (UrbanNav-HK_Whampoa-20210521_sensors.csv) to 'cv_set' dataframe

# uncomment and compelete below
cv_set = pd.read_csv('UrbanNav-HK_Whampoa-20210521_sensors.csv')

# TODO: Load UrbanNav-HK_TST-20210517_sensors.csv to 'test_set1' dataframe
test_set1 = pd.read_csv('UrbanNav-HK_TST-20210517_sensors.csv')

# TODO: Load 2020-03-14-16-45-35.csv to 'test_set2' dataframe
test_set2 = pd.read_csv('2020-03-14-16-45-35.csv')

For the *cv_set*, look at the dataframe head and info, and print the list of all columns. The *test_set1* and *test_set2* have the same schema as *cv_set*, however, have different number of rows (samples).

In [3]:
# TODO: Display first two rows of cv_set
# (1 mark)

print(cv_set.head(2))

     ax_sum    ay_sum    az_sum    gx_sum    gy_sum    gz_sum     a_sum  \
0 -0.003929 -0.093081  9.789349 -0.002649  0.001406 -0.001391  9.797418   
1 -0.000966 -0.092417  9.789095 -0.001806  0.000103 -0.001943  9.798815   

      g_sum        ax_var        ay_var  ...     g_min    ax_max    ay_max  \
0  0.022554  8.046094e-07  1.319665e-07  ...  0.000003  0.001496  0.000544   
1  0.021731  1.032619e-06  1.073067e-07  ...  0.000006  0.001865  0.000615   

     az_max    gx_max    gy_max    gz_max     a_max     g_max  motion_state  
0  0.024992  0.000110  0.000075  0.000078  0.024997  0.000187          rest  
1  0.024828  0.000124  0.000098  0.000090  0.024829  0.000144          rest  

[2 rows x 33 columns]


In [4]:
# TODO: Load the info method of cv_set
# (1 mark)

cv_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1534 entries, 0 to 1533
Data columns (total 33 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ax_sum        1534 non-null   float64
 1   ay_sum        1534 non-null   float64
 2   az_sum        1534 non-null   float64
 3   gx_sum        1534 non-null   float64
 4   gy_sum        1534 non-null   float64
 5   gz_sum        1534 non-null   float64
 6   a_sum         1534 non-null   float64
 7   g_sum         1534 non-null   float64
 8   ax_var        1534 non-null   float64
 9   ay_var        1534 non-null   float64
 10  az_var        1534 non-null   float64
 11  gx_var        1534 non-null   float64
 12  gy_var        1534 non-null   float64
 13  gz_var        1534 non-null   float64
 14  a_var         1534 non-null   float64
 15  g_var         1534 non-null   float64
 16  ax_min        1534 non-null   float64
 17  ay_min        1534 non-null   float64
 18  az_min        1534 non-null 

In [5]:
# TODO: Display the number of rows and columns  of all three datasets
# (1 mark)

print(f"'cv_set' has {cv_set.shape[0]} rows and {cv_set.shape[1]} columns")
print(f"'test_set1' has {test_set1.shape[0]} rows and {test_set1.shape[1]} columns")
print(f"'test_set2' has {test_set2.shape[0]} rows and {test_set2.shape[1]} columns")

'cv_set' has 1534 rows and 33 columns
'test_set1' has 783 rows and 33 columns
'test_set2' has 297 rows and 33 columns


In [6]:
# TODO: Display all columns in cv_set
# (1 mark)

print(cv_set.columns.tolist())

['ax_sum', 'ay_sum', 'az_sum', 'gx_sum', 'gy_sum', 'gz_sum', 'a_sum', 'g_sum', 'ax_var', 'ay_var', 'az_var', 'gx_var', 'gy_var', 'gz_var', 'a_var', 'g_var', 'ax_min', 'ay_min', 'az_min', 'gx_min', 'gy_min', 'gz_min', 'a_min', 'g_min', 'ax_max', 'ay_max', 'az_max', 'gx_max', 'gy_max', 'gz_max', 'a_max', 'g_max', 'motion_state']


In all three dataframes, target vector is stored in the *motion_state* column. All other columns are features.

___

### 1.2 - Grid Search with Cross Validation (18 Marks)

___

#### 1.2.1 - Extract all features and target from cv_set

___

From the *cv_set*, load the target vector stored in the *motion_state* column into a new variable *y*

In [7]:
# TODO - Load target vector into y
# (1 mark)

y = cv_set['motion_state']

From *cv_set*, load all features, that is all columns except for the target vector, in a new dataframe variable *X*

In [8]:
# TODO - Load all features from cv_set to X
# (2 marks)

X = cv_set.drop(columns=['motion_state'])
X.head(2)

,ax_sum,ay_sum,az_sum,gx_sum,gy_sum,gz_sum,a_sum,g_sum,ax_var,ay_var,...,a_min,g_min,ax_max,ay_max,az_max,gx_max,gy_max,gz_max,a_max,g_max
0,-0.003929,-0.093081,9.789349,-0.002649,0.001406,-0.001391,9.797418,0.022554,8.046094e-07,1.319665e-07,...,0.023871,0.000003,0.001496,0.000544,0.024992,0.000110,0.000075,0.000078,0.024997,0.000187
1,-0.000966,-0.092417,9.789095,-0.001806,0.000103,-0.001943,9.798815,0.021731,1.032619e-06,1.073067e-07,...,0.024153,0.000006,0.001865,0.000615,0.024828,0.000124,0.000098,0.000090,0.024829,0.000144


___

#### 1.2.2 - Configure and Perform Grid Search with Cross-Validation

___

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, Normalizer
from sklearn.model_selection import GridSearchCV

The dataset *(X, y)*, must be split into training/validation *(X_train, y_train)* and testing *(X_test, y_test)* sets. Use the `train_test_split` function from sklearn to create the split, where  **30%** of data is used for testing. Set the `random_state=42`, and `stratify=y`.  

In [10]:
# TODO - Perform test/train split
# (1 mark)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

Initialize a `Pipeline`, consisting of a *scaler* and a *classifer*, set the default values to *StandardScaler()*, and *SVC()*, respectively.

In [11]:
# TODO - Initialize Pipeline into variable pipe
# (2 marks)

pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', SVC())
])

Define the grid search configuration (*param_grid*) for training and hyper parameter tuning of the *support vector classifier*. Tune, the hyper-parameter *C*, over the interval `C=[0.01, 0.1, 1 ,10,100]`, *gamma*, over the interval `gamma=[0.001, 0.01, 0.1,1]`, and use the *radial basis function* as the kernel. For the *scaler* parameter, test with the `StandardScaler`, `RobustScaler`, and `None` options.

In [12]:
# TODO - Setup grid set
# (4 marks)

param_grid = {
    'scaler': [StandardScaler(), RobustScaler(), None],
    'classifier__C': [0.01, 0.1, 1, 10, 100],
    'classifier__gamma': [0.001, 0.01, 0.1, 1],
    'classifier__kernel': ['rbf']
}

In [13]:
# Perform the grid search to train model and tune hyper parameters
grid = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy', return_train_score=True)
grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('classifier', SVC())]),
             param_grid={'classifier__C': [0.01, 0.1, 1, 10, 100],
                         'classifier__gamma': [0.001, 0.01, 0.1, 1],
                         'classifier__kernel': ['rbf'],
                         'scaler': [StandardScaler(), RobustScaler(), None]},
             return_train_score=True, scoring='accuracy')

___

#### 1.2.3 - Visualize Grid Search Results

___

Print the results from the grid search. The following outputs must be printed.
* Best parameter set
* Best cross-validation train score
* Best cross-validation test score
* Test set accuracy

In [14]:
# TODO: Print the results from the grid search
# (2 marks)

print(f"Best params:\n{grid.best_params_}\n")
print(f"Best cross-validation train score: {grid.best_score_:.2f}")
print(f"Test-set score: {grid.score(X_test, y_test):.2f}")

Best params:
{'classifier__C': 100, 'classifier__gamma': 0.01, 'classifier__kernel': 'rbf', 'scaler': RobustScaler()}

Best cross-validation train score: 0.95
Test-set score: 0.96


Display the confusion matrix and print the classification report for the trained model. Use the *(X_test, y_test)* dataset. **Hint** - You can use your code from assignment 3.

In [15]:
# TODO - Display confusion matrix and print classification report for (X_test, y_test)
# (2 marks)

from sklearn.metrics import classification_report, confusion_matrix

# Display confusion matrix and print classification report for (X_test, y_test)
y_pred = grid.predict(X_test)
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Confusion Matrix:
 [[153   2   0]
 [ 10 192   5]
 [  0   3  96]]

Classification Report:
               precision    recall  f1-score   support

        rest       0.94      0.99      0.96       155
    straight       0.97      0.93      0.95       207
        turn       0.95      0.97      0.96        99

    accuracy                           0.96       461
   macro avg       0.95      0.96      0.96       461
weighted avg       0.96      0.96      0.96       461



___

#### 1.2.4 - Questions

1. Comment on the fit of the model to the dataset? (2 marks)
2. In the grid search, are the intervals used for *C* and *gamma* hyperparameters suitable? Why or why not? (2 marks)

#### 1.2.4 - Answers
1. Fit of the Model:
- The model fits the data well, achieving **96% accuracy** with strong precision and recall across all classes. This shows the SVC model captures class patterns effectively.
---
2. Suitability of Hyperparameter Intervals:
- The chosen ranges for **C** and **gamma** were appropriate. They provided sufficient flexibility to find optimal values, balancing complexity and smooth decision boundaries.  

___

### 1.3 - Model Evaluation on Test Datasets 1 and 2 (9 Marks)

___

#### 1.3.1 - Extract all features and targets from test datasets

Next, the model trained in 1.2 will be tested on independent datasets, that is *test_set1* and *test_set2*. <br>
The test_sets must be seperated into features (Xs) and targets (ys).

In [16]:
# TODO - Load the target vector from test_set1 into y1, and all features from test_set1 into X1
# (1 marks)

y1 = test_set1['motion_state']
X1 = test_set1.drop(columns=['motion_state'])
X1.head(2)

,ax_sum,ay_sum,az_sum,gx_sum,gy_sum,gz_sum,a_sum,g_sum,ax_var,ay_var,...,a_min,g_min,ax_max,ay_max,az_max,gx_max,gy_max,gz_max,a_max,g_max
0,0.264465,0.089904,9.786930,-0.002305,0.000218,-0.001143,9.801661,0.025634,0.000001,7.749919e-08,...,0.024056,0.000003,0.002506,0.000845,0.024884,0.000155,0.000078,0.000069,0.024886,0.000170
1,0.264300,0.088882,9.788127,-0.001865,-0.000523,-0.001999,9.802481,0.025218,0.000001,9.243000e-08,...,0.024084,0.000007,0.002427,0.000949,0.024881,0.000201,0.000095,0.000076,0.024883,0.000201


In [17]:
# TODO - Load the target vector from test_set2 into y2, and all features from test_set2 into X2

y2 = test_set2['motion_state']
X2 = test_set2.drop(columns=['motion_state'])
X2.head(2)

,ax_sum,ay_sum,az_sum,gx_sum,gy_sum,gz_sum,a_sum,g_sum,ax_var,ay_var,...,a_min,g_min,ax_max,ay_max,az_max,gx_max,gy_max,gz_max,a_max,g_max
0,-0.046400,0.305907,9.791451,-0.002184,0.001311,-0.001901,9.800683,0.012437,0.000002,3.020959e-07,...,0.048406,0.000007,0.002575,0.002993,0.049701,0.000094,0.000115,0.000080,0.049764,0.000142
1,-0.050073,0.311947,9.785619,-0.003369,0.001898,-0.002051,9.797741,0.013805,0.000003,6.859937e-07,...,0.047696,0.000013,0.003880,0.004106,0.050477,0.000140,0.000140,0.000083,0.050763,0.000151


____

#### 1.3.2 - Visualize Results

For each test dataset, print the test set accuracy score.

In [18]:
# TODO - Print test score accuracies for test_set1 and test_set2
# (1 marks)

print(f"test_set1 score: {grid.score(X1, y1):.2f}")
print(f"test_set2 score: {grid.score(X2, y2):.2f}")

test_set1 score: 0.92
test_set2 score: 0.25


Display the confusion matrix and print the classification for test_set 1 *(X1, y1)*

In [19]:
# TODO - Display confusion matrix and print classification report for (X1, y1)
# (1 marks)

y1_pred = grid.predict(X1)
print("Confusion Matrix for test_set1:\n", confusion_matrix(y1, y1_pred))
print("\nClassification Report for test_set1:\n", classification_report(y1, y1_pred))

Confusion Matrix for test_set1:
 [[200  31   0]
 [  7 387  17]
 [  0   7 134]]

Classification Report for test_set1:
               precision    recall  f1-score   support

        rest       0.97      0.87      0.91       231
    straight       0.91      0.94      0.93       411
        turn       0.89      0.95      0.92       141

    accuracy                           0.92       783
   macro avg       0.92      0.92      0.92       783
weighted avg       0.92      0.92      0.92       783



Display the confusion matrix and print the classification for test_set 2 *(X2, y2)*

In [20]:
# TODO - Display confusion matrix and print classification report for (X2, y2)

y2_pred = grid.predict(X2)
print("Confusion Matrix for test_set2:\n", confusion_matrix(y2, y2_pred))

# Updated classification report with zero_division parameter to avoid the warrning of this situation could occur if the model is not generalizing well to certain classes,
# possibly because those classes are rare or the model was not adequately trained to recognize them.
print("\nClassification Report for test_set2:\n", classification_report(y2, y2_pred, zero_division=0))

Confusion Matrix for test_set2:
 [[  0   0  16]
 [  0   0 206]
 [  0   0  75]]

Classification Report for test_set2:
               precision    recall  f1-score   support

        rest       0.00      0.00      0.00        16
    straight       0.00      0.00      0.00       206
        turn       0.25      1.00      0.40        75

    accuracy                           0.25       297
   macro avg       0.08      0.33      0.13       297
weighted avg       0.06      0.25      0.10       297



___

#### 1.3.3 - Questions

**Based on results so far**
1. Comment on the model's performances on the datasets *(X_test, y_test)*, *(X1, y1)*, *(X2, y2)*? A significant difference in performance should be observed, what is this an example of? (3 marks)
1. Provide and explain potential causes for the observed behaviour in performance? (3 marks)

#### 1.3.3 - Answers
1. The model performs well on **cv_set (96% accuracy)** and **test_set1 (92% accuracy)**, but fails on **test_set2 (25% accuracy)**, indicating a potential domain shift between training and test datasets.
2. The poor performance on *test_set2* suggests data distribution differences or overfitting to *cv_set*, with the model struggling to generalize to unseen variations in *test_set2*.

___

## Section 2 - Classification Using *Selected Features* Derived From Inertial Sensor Data (12 Marks)

In this second section, an *SVC* model will be trained and tuned using grid search, however, only *selected features* will be used.

___

### 2.1 Grid Search with Cross Validation (5 Marks)

___

#### 2.1.1 Extract *selected* features and target from *cv_set*

From the *cv_set* dataframe, extract the selected features, listed below, and load them in a new dataframe *X*.
* *ax_var* - variance of accelerometer readings in x
* *ay_var* - variance of accelerometer readings in y
* *az_sum* - sum of accelerometer z readings
* *a_sum* - sum of the norm of accelerometer x, y, z readings
* *gz_var* - variance of gyroscope reading in z

In [21]:
# TODO - Load selected features from cv_set to X
# (1 marks)

X = cv_set[['ax_var', 'ay_var', 'az_sum', 'a_sum', 'gz_var']]
X.head(2)

,ax_var,ay_var,az_sum,a_sum,gz_var
0,8.046094e-07,1.319665e-07,9.789349,9.797418,7.446916e-10
1,1.032619e-06,1.073067e-07,9.789095,9.798815,6.983481e-10


In [22]:
# TODO - Load motion_state column (target vector) into y

y = cv_set['motion_state']

___

#### 2.1.2 - Configure and Perform Grid Search with Cross-Validation

Follow the steps from **1.2.2** and perform model training and tuning using the selected feature set.

In [ ]:
# TODO - Perform model training and tuning using `GridSearchCV`

# Split the dataset into training and testing sets
# Perform test/train split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Initialize a Pipeline with StandardScaler and SVC
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('svc', SVC())
])

# Define the grid search configuration
param_grid = {
    'scaler': [StandardScaler(), RobustScaler(), None],
    'svc__C': [0.01, 0.1, 1, 10, 100],
    'svc__gamma': [0.001, 0.01, 0.1, 1],
    'svc__kernel': ['rbf']
}

# Perform the grid search to train model and tune hyperparameters
grid = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy', return_train_score=True)
grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('svc', SVC())]),
             param_grid={'scaler': [StandardScaler(), RobustScaler(), None],
                         'svc__C': [0.01, 0.1, 1, 10, 100],
                         'svc__gamma': [0.001, 0.01, 0.1, 1],
                         'svc__kernel': ['rbf']},
             return_train_score=True, scoring='accuracy')

___

#### 2.1.3 - Visualize Grid Search Results

Print the results from the grid search. The following outputs must be printed.
* Best parameter set
* Best cross-validation train score
* Best cross-validation test score
* Test set accuracy

In [ ]:
# TODO: Print the results from the grid search

print("Best params:\n{}\n".format(grid.best_params_))
print("Best cross-validation train score: {:.2f}".format(grid.best_score_))
print("Test-set score: {:.2f}".format(grid.score(X_test, y_test)))

Best params:
{'scaler': RobustScaler(), 'svc__C': 100, 'svc__gamma': 0.01, 'svc__kernel': 'rbf'}

Best cross-validation train score: 0.82
Test-set score: 0.83


Display the confusion matrix and print the classification report for the trained model using the *test* split.

In [ ]:
# TODO - Display confusion matrix and print classification reports

y_pred = grid.predict(X_test)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", class_report)

Confusion Matrix:
 [[154   1   0]
 [ 20 173  14]
 [  1  44  54]]

Classification Report:
               precision    recall  f1-score   support

        rest       0.88      0.99      0.93       155
    straight       0.79      0.84      0.81       207
        turn       0.79      0.55      0.65        99

    accuracy                           0.83       461
   macro avg       0.82      0.79      0.80       461
weighted avg       0.82      0.83      0.82       461



___

#### 2.1.4 - Question


1. Consider the differences between the performance on the training/cross-validation dataset for section 1 (1.2.3) and section 2 (2.1.3). Which model is a better fit based on *just these two evaluations*? (4 marks)

#### 2.1.4 Answer
1. The model from **Section 1 is a better fit**, achieving **96% accuracy** compared to **83% in Section 2**. It showed consistently better precision and recall across all classes, suggesting that using all features led to more accurate classifications than the selected features in Section 2

___

### 2.2 - Model Evaluation on Test Datasets 1 and 2 (7 Marks)

___

#### 2.2.1 - Extract selected features and targets from test datasets

The model trained in 2.1 is to be tested on the independent datasets, that is test_set1 and test_set2.
Seperate the testsets into the selected features and target vectors *(X1, y1)* and (X2,)

In [ ]:
# TODO - Load the target vector from test_set1 into y1, and the selected features from test_set1 into X1
# (1 marks)

y1 = test_set1['motion_state']
X1 = test_set1[['ax_var', 'ay_var', 'az_sum', 'a_sum', 'gz_var']]
X1.head(2)

,ax_var,ay_var,az_sum,a_sum,gz_var
0,0.000001,7.749919e-08,9.786930,9.801661,7.727190e-10
1,0.000001,9.243000e-08,9.788127,9.802481,8.871433e-10


In [ ]:
# TODO - Load the target vector from test_set2 into y2, and the selected features from test_set2 into X2

y2 = test_set2['motion_state']
X2 = test_set2[['ax_var', 'ay_var', 'az_sum', 'a_sum', 'gz_var']]
X2.head(2)

,ax_var,ay_var,az_sum,a_sum,gz_var
0,0.000002,3.020959e-07,9.791451,9.800683,1.405038e-09
1,0.000003,6.859937e-07,9.785619,9.797741,1.164381e-09


___

#### 2.2.2 - Visualize Results

For each test dataset, print the test set accuracy score.

In [ ]:
# TODO - Print test score accuracies for test_set1 and test_set2

print("test_set1 score: {:.2f}".format(grid.score(X1, y1)))
print("test_set2 score: {:.2f}".format(grid.score(X2, y2)))

test_set1 score: 0.83
test_set2 score: 0.84


Display the confusion matrix and print the classification for test_set 1 *(X1, y1)*

In [ ]:
# TODO - Display confusion matrix and print classification report for (X1, y1)

y1_pred = grid.predict(X1)
conf_matrix1 = confusion_matrix(y1, y1_pred)
class_report1 = classification_report(y1, y1_pred)
print("\nConfusion Matrix for test_set1:\n", conf_matrix1)
print("\nClassification Report for test_set1:\n", class_report1)


Confusion Matrix for test_set1:
 [[229   2   0]
 [ 26 336  49]
 [  0  57  84]]

Classification Report for test_set1:
               precision    recall  f1-score   support

        rest       0.90      0.99      0.94       231
    straight       0.85      0.82      0.83       411
        turn       0.63      0.60      0.61       141

    accuracy                           0.83       783
   macro avg       0.79      0.80      0.80       783
weighted avg       0.83      0.83      0.83       783



Display the confusion matrix and print the classification for test_set 2 *(X2, y2)*

In [ ]:
# TODO - Display confusion matrix and print classification report for (X2, y2)

y2_pred = grid.predict(X2)
conf_matrix2 = confusion_matrix(y2, y2_pred)
class_report2 = classification_report(y2, y2_pred)
print("\nConfusion Matrix for test_set2:\n", conf_matrix2)
print("\nClassification Report for test_set2:\n", class_report2)


Confusion Matrix for test_set2:
 [[ 15   1   0]
 [  0 178  28]
 [  0  18  57]]

Classification Report for test_set2:
               precision    recall  f1-score   support

        rest       1.00      0.94      0.97        16
    straight       0.90      0.86      0.88       206
        turn       0.67      0.76      0.71        75

    accuracy                           0.84       297
   macro avg       0.86      0.85      0.85       297
weighted avg       0.85      0.84      0.84       297



___

#### 2.2.3 Questions

1. For section 2, compare between the model's performance on the training/validation (2.1) set to the the performance on the two testing sets (2.2). How does the model fit? (3 marks)
1. Compare the models' performances and fits between section 1 and section 2, and consider all three datasets. (3 marks)

#### 2.2.3 Answers
1. The model performed well on the training/validation set, **achieving 83% accuracy**, but its performance on the test sets showed more variation. For **test_set1**, the model **achieved 83% accuracy**, but struggled with **"turn" class (61% recall)**, whereas for **test_set2**, it performed slightly better with **84% accuracy**, showing more consistent results across all classes.
2. The model in **Section 1** consistently outperformed **Section 2** across all datasets, with **Section 1 achieving 96% accuracy** on the training data, while **Section 2's best accuracy was 83%**. On the test datasets, Section 1's model showed stronger performance overall, especially for the "turn" class, whereas Section 2's model struggled more, particularly with the **"turn" class in test_set1 (60% recall).**

____

## Section 3 - Classification Using *Group Selected Features* From Inertial Sensor Data (10 Marks)

Once again, an *SVC* model is to be trained and tuned using grid search. However, you must select which features to use from the ones provided in *cv_set*.  

____

### 3.1 Grid Search with Cross Validation (4 Marks)

___

#### 3.1.1 Extract *Group Selected* features and target from *cv_set*

**Question**
1. Please list the features that you have selected and explain *why you have chosen* these features? (4 marks)


**Answer**
1. We have selected the following features based on their relevance to the classification task:

- ax_var (variance of accelerometer readings in x): Captures variability in the x-axis, which could relate to changes in motion.
- ay_var (variance of accelerometer readings in y): Similar to ax_var, capturing movement along the y-axis.
- az_sum (sum of accelerometer z readings): Represents total movement along the z-axis, which can indicate dynamic motion.
- gz_var (variance of gyroscope readings in z): Reflects rotational movements around the z-axis. These features provide a combination of variability and total movement, which are important for distinguishing motion states.   

In [ ]:
# TODO - Load group selected features from cv_set to X

X = cv_set[['ax_var', 'ay_var', 'az_sum', 'gz_var']]
X.head(2)

,ax_var,ay_var,az_sum,gz_var
0,8.046094e-07,1.319665e-07,9.789349,7.446916e-10
1,1.032619e-06,1.073067e-07,9.789095,6.983481e-10


In [ ]:
# TODO - Load motion_state column (target vector) into y

y = cv_set['motion_state']

___

#### 3.1.2 - Configure and Perform Grid Search with Cross-Validation

Follow the steps from **1.2.2** and perform model training and tuning using the group selected feature set.

In [ ]:
# TODO - Perform model training and tuning using `GridSearchCV`

# Step 1: Split the data into training and testing sets (if not already done)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 2: Define the parameter grid for the grid search
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['rbf'],
    'gamma': ['scale', 'auto'],
}

# Step 3: Create a Support Vector Classifier (SVC) model
svc = SVC()

# Step 4: Perform Grid Search with Cross-Validation
grid_search = GridSearchCV(estimator=svc, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Step 5: Fit the grid search to the training data
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=SVC(), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10], 'gamma': ['scale', 'auto'],
                         'kernel': ['rbf']},
             scoring='accuracy')

___

#### 3.1.3 - Visualize Grid Search Results

Print the results from the grid search. The following outputs must be printed.
* Best parameter set
* Best cross-validation train score
* Best cross-validation test score
* Test set accuracy

In [ ]:
# TODO: Print the results from the grid search

print("Best params:\n{}\n".format(grid_search.best_params_))
print("Best cross-validation train score: {:.2f}".format(grid_search.best_score_))
print("Best cross-validation test score: {:.2f}".format(grid_search.best_estimator_.score(X_test, y_test)))
print("Test-set score: {:.2f}".format(grid_search.score(X_test, y_test)))

Best params:
{'C': 0.1, 'gamma': 'scale', 'kernel': 'rbf'}

Best cross-validation train score: 0.45
Best cross-validation test score: 0.45
Test-set score: 0.45


Display the confusion matrix and print the classification report for the trained model using the *test* split.

In [ ]:
# TODO - Display confusion matrix and print classification report


y_pred = grid_search.predict(X_test)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred, zero_division=0)

print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", class_report)

Confusion Matrix:
 [[  0 104   0]
 [  0 137   0]
 [  0  66   0]]
Classification Report:
               precision    recall  f1-score   support

        rest       0.00      0.00      0.00       104
    straight       0.45      1.00      0.62       137
        turn       0.00      0.00      0.00        66

    accuracy                           0.45       307
   macro avg       0.15      0.33      0.21       307
weighted avg       0.20      0.45      0.28       307



___

### 3.2 - Model Evaluation on Test Datasets 1 and 2 (6 Marks)

___

#### 3.2.1 - Extract group selected features and targets from test datasets

The model trained in 3.1 is to be tested on the independent datasets, that is test_set1 and test_set2.
Seperate the testsets into group selected features and target vectors *(X1, y1)* and *(X2,y2)*

In [ ]:
# TODO - Load the target vector from test_set1 into y1, and the group selected features from test_set1 into X1

y1 = test_set1['motion_state']
X1 = test_set1[['ax_var', 'ay_var', 'az_sum', 'gz_var']]
X1.head(2)

,ax_var,ay_var,az_sum,gz_var
0,0.000001,7.749919e-08,9.786930,7.727190e-10
1,0.000001,9.243000e-08,9.788127,8.871433e-10


In [ ]:
# TODO - Load the target vector from test_set2 into y2, and the group selected features from test_set2 into X2

y2 = test_set2['motion_state']
X2 = test_set2[['ax_var', 'ay_var', 'az_sum', 'gz_var']]
X2.head(2)

,ax_var,ay_var,az_sum,gz_var
0,0.000002,3.020959e-07,9.791451,1.405038e-09
1,0.000003,6.859937e-07,9.785619,1.164381e-09


___

#### 3.2.2 - Visualize Results

For each test dataset, print the test set accuracy score.

In [ ]:
# TODO - Print test score accuracies for test_set1 and test_set2

print("test_set1 score: {:.2f}".format(grid_search.score(X1, y1)))
print("test_set2 score: {:.2f}".format(grid_search.score(X2, y2)))

test_set1 score: 0.52
test_set2 score: 0.69


Display the confusion matrix and print the classification for test_set 1 *(X1, y1)*

In [ ]:
# TODO - Display confusion matrix and print classification report for (X1, y1)

y1_pred = grid_search.predict(X1)
print("test_set1 Confusion Matrix:\n", confusion_matrix(y1, y1_pred))
print("test_set1 Classification Report:\n", classification_report(y1, y1_pred, zero_division=0))

test_set1 Confusion Matrix:
 [[  0 231   0]
 [  0 411   0]
 [  0 141   0]]
test_set1 Classification Report:
               precision    recall  f1-score   support

        rest       0.00      0.00      0.00       231
    straight       0.52      1.00      0.69       411
        turn       0.00      0.00      0.00       141

    accuracy                           0.52       783
   macro avg       0.17      0.33      0.23       783
weighted avg       0.28      0.52      0.36       783



Display the confusion matrix and print the classification for test_set 2 *(X2, y2)*

In [ ]:
# TODO - Display confusion matrix and print classification report for (X2, y2)

y2_pred = grid_search.predict(X2)
print("test_set2 Confusion Matrix:\n", confusion_matrix(y2, y2_pred))
print("test_set2 Classification Report:\n", classification_report(y2, y2_pred, zero_division=0))

test_set2 Confusion Matrix:
 [[  0  16   0]
 [  0 206   0]
 [  0  75   0]]
test_set2 Classification Report:
               precision    recall  f1-score   support

        rest       0.00      0.00      0.00        16
    straight       0.69      1.00      0.82       206
        turn       0.00      0.00      0.00        75

    accuracy                           0.69       297
   macro avg       0.23      0.33      0.27       297
weighted avg       0.48      0.69      0.57       297



___

#### 3.2.3 Questions

1. Based on the observations across all three datasets, compare the performance of the model trained in this section to: (3 marks)
    1. Model trained in Section 1 using *all features*.
    1. Model trained in Section 2 using *selected features*.

2. Please proivde potential explanations for the differences observed in models' performances across the three sections. (3 marks)

#### 3.2.3 Answers
1. Comparison of Model Performance Across Sections:

- Section 1 (All Features): Achieved 96% accuracy with good precision and recall, especially for "rest" and "straight" classes. The "turn" class showed slightly lower performance.

- Section 2 (Selected Features): Performance dropped to 83% accuracy with lower recall for "turn" and some degradation in "straight," but still decent on "rest."

- Section 3 (Group Selected Features): Performed the worst with 45% accuracy in training and 52-69% on the test sets. The model failed to predict the "turn" class effectively.
---
2. Explanations for Performance Differences:

- Section 1 vs. Section 2: The drop from Section 1 to Section 2 is likely due to removing features that were useful for class prediction, particularly for the "turn" class.

- Section 2 vs. Section 3: Section 3 performed poorly due to limited group-selected features, which likely didn’t capture enough predictive information, especially for "turn."

- Class Imbalance: The "turn" class underperformed in all sections, possibly due to its smaller representation in the data.

____

## Section 4 - Classification Using Principal Components of Features From Inertial Sensor Data (10 Marks)

____

### 4.1 Principal Component Analysis (PCA) Search with Cross Validation (4 Marks)

___

#### 4.1.1 Extract *ALL* features and target from *cv_set*

In [ ]:
# TODO - Load all features from cv_set to X

X = cv_set.drop(columns=['motion_state'])
X.head(2)

,ax_sum,ay_sum,az_sum,gx_sum,gy_sum,gz_sum,a_sum,g_sum,ax_var,ay_var,...,a_min,g_min,ax_max,ay_max,az_max,gx_max,gy_max,gz_max,a_max,g_max
0,-0.003929,-0.093081,9.789349,-0.002649,0.001406,-0.001391,9.797418,0.022554,8.046094e-07,1.319665e-07,...,0.023871,0.000003,0.001496,0.000544,0.024992,0.000110,0.000075,0.000078,0.024997,0.000187
1,-0.000966,-0.092417,9.789095,-0.001806,0.000103,-0.001943,9.798815,0.021731,1.032619e-06,1.073067e-07,...,0.024153,0.000006,0.001865,0.000615,0.024828,0.000124,0.000098,0.000090,0.024829,0.000144


In [ ]:
# TODO - Load motion_state column (target vector) into y

y = cv_set['motion_state']

___

#### 4.1.2 - Configure and Perform Grid Search with Cross-Validation

Similar to the steps from **1.2.2** and perform model training and tuning for an `SVC` model. Perform the `train_test_split`, initiate a `Pipeline`, then define the *Parameters* for `GridSearchCV`. Finally, perfrom the grid search.

In [ ]:
# TODO - Perform model training and tuning using `GridSearchCV`

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In the `Pipeline` add an entry *'pca'* to apply `PCA`.

In [ ]:
# Perform any missing imports
from sklearn.decomposition import PCA

# TODO - Initiate the pipeline
# (1 marks)

pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA()),
    ('svc', SVC())
])


Similar to previous sections, define the grid search configuration (*param_grid*). Add *n_components* hyper parameter tuning for PCA, and search over the values `n_components=[4,7,11,14,16]`.  

In [ ]:
# TODO - Define the parameter grid for your model
# (3 marks)

param_grid = {
    'pca__n_components': [4, 7, 11, 14, 16],
    'svc__C': [0.1, 1, 10],
    'svc__kernel': ['linear', 'rbf'],
}

In [ ]:
# Perform the grid search to train model and tune hyper parameters
grid = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy', return_train_score=True)
grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('pca', PCA()), ('svc', SVC())]),
             param_grid={'pca__n_components': [4, 7, 11, 14, 16],
                         'svc__C': [0.1, 1, 10],
                         'svc__kernel': ['linear', 'rbf']},
             return_train_score=True, scoring='accuracy')

___

#### 4.1.3 - Visualize Grid Search Results

Print the results from the grid search. The following outputs must be printed.
* Best parameter set
* Best cross-validation train score
* Best cross-validation test score
* Test set accuracy

In [ ]:
# TODO: Print the results from the grid search

print("Best params:\n{}\n".format(grid.best_params_))
print("Best cross-validation train score: {:.2f}".format(grid.best_score_))
print("Best cross-validation validation score: {:.2f}".format(grid.cv_results_['mean_test_score'][grid.best_index_]))
print("Test-set score: {:.2f}".format(grid.score(X_test, y_test)))

Best params:
{'pca__n_components': 14, 'svc__C': 10, 'svc__kernel': 'linear'}

Best cross-validation train score: 0.94
Best cross-validation validation score: 0.94
Test-set score: 0.95


Display the confusion matrix and print the classification report for the trained model using the *test* split.

In [ ]:
# TODO - Display confusion matrix and print classification report

y_pred = grid.predict(X_test)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred, zero_division=0)

print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", class_report)

Confusion Matrix:
 [[149   2   0]
 [ 14 195   4]
 [  0   3  94]]
Classification Report:
               precision    recall  f1-score   support

        rest       0.91      0.99      0.95       151
    straight       0.97      0.92      0.94       213
        turn       0.96      0.97      0.96        97

    accuracy                           0.95       461
   macro avg       0.95      0.96      0.95       461
weighted avg       0.95      0.95      0.95       461



___

### 4.2 - Model Evaluation on Test Datasets 1 and 2 (6 Marks)

___

#### 4.2.1 - Extract group selected features and targets from test datasets

In [ ]:
# TODO - Load the target vector from test_set1 into y1, and all features from test_set1 into X1

y1 = test_set1['motion_state']
X1 = test_set1.drop(columns=['motion_state'])
X1.head(2)

,ax_sum,ay_sum,az_sum,gx_sum,gy_sum,gz_sum,a_sum,g_sum,ax_var,ay_var,...,a_min,g_min,ax_max,ay_max,az_max,gx_max,gy_max,gz_max,a_max,g_max
0,0.264465,0.089904,9.786930,-0.002305,0.000218,-0.001143,9.801661,0.025634,0.000001,7.749919e-08,...,0.024056,0.000003,0.002506,0.000845,0.024884,0.000155,0.000078,0.000069,0.024886,0.000170
1,0.264300,0.088882,9.788127,-0.001865,-0.000523,-0.001999,9.802481,0.025218,0.000001,9.243000e-08,...,0.024084,0.000007,0.002427,0.000949,0.024881,0.000201,0.000095,0.000076,0.024883,0.000201


In [ ]:
# TODO - Load the target vector from test_set2 into y2, and all features from test_set2 into X2

y2 = test_set2['motion_state']
X2 = test_set2.drop(columns=['motion_state'])
X2.head(2)

,ax_sum,ay_sum,az_sum,gx_sum,gy_sum,gz_sum,a_sum,g_sum,ax_var,ay_var,...,a_min,g_min,ax_max,ay_max,az_max,gx_max,gy_max,gz_max,a_max,g_max
0,-0.046400,0.305907,9.791451,-0.002184,0.001311,-0.001901,9.800683,0.012437,0.000002,3.020959e-07,...,0.048406,0.000007,0.002575,0.002993,0.049701,0.000094,0.000115,0.000080,0.049764,0.000142
1,-0.050073,0.311947,9.785619,-0.003369,0.001898,-0.002051,9.797741,0.013805,0.000003,6.859937e-07,...,0.047696,0.000013,0.003880,0.004106,0.050477,0.000140,0.000140,0.000083,0.050763,0.000151


___

#### 4.2.2 - Visualize Results

In [ ]:
# TODO - Print test score accuracies for test_set1 and test_set2

best_model = grid.best_estimator_

# Test set 1
y1_pred = best_model.predict(X1)
print("test_set1 score: {:.2f}".format(best_model.score(X1, y1)))

# Test set 2
y2_pred = best_model.predict(X2)
print("test_set2 score: {:.2f}".format(best_model.score(X2, y2)))

test_set1 score: 0.92
test_set2 score: 0.92


In [ ]:
# TODO - Display confusion matrix and print classification report for (X1, y1)

y1_pred = grid.predict(X1)
print("test_set1 Confusion Matrix:\n", confusion_matrix(y1, y1_pred))
print("test_set1 Classification Report:\n", classification_report(y1, y1_pred, zero_division=0))

test_set1 Confusion Matrix:
 [[225   6   0]
 [ 20 379  12]
 [  2  19 120]]
test_set1 Classification Report:
               precision    recall  f1-score   support

        rest       0.91      0.97      0.94       231
    straight       0.94      0.92      0.93       411
        turn       0.91      0.85      0.88       141

    accuracy                           0.92       783
   macro avg       0.92      0.92      0.92       783
weighted avg       0.92      0.92      0.92       783



Display the confusion matrix and print the classification for test_set 2 *(X2, y2)*

In [ ]:
# TODO - Display confusion matrix and print classification report for (X2, y2)

y2_pred = grid.predict(X2)
print("test_set2 Confusion Matrix:\n", confusion_matrix(y2, y2_pred))
print("test_set2 Classification Report:\n", classification_report(y2, y2_pred, zero_division=0))

test_set2 Confusion Matrix:
 [[ 14   2   0]
 [  5 200   1]
 [  0  16  59]]
test_set2 Classification Report:
               precision    recall  f1-score   support

        rest       0.74      0.88      0.80        16
    straight       0.92      0.97      0.94       206
        turn       0.98      0.79      0.87        75

    accuracy                           0.92       297
   macro avg       0.88      0.88      0.87       297
weighted avg       0.92      0.92      0.92       297



___

#### 4.2.3 - Questions

1. Based on the observations across all three datasets, compare the performance of the model with PCA, trained in this section to: (4 marks)
    * Model trained in Section 1 using *all features*.
    * Model trained in Section 2 using *selected features*.
    * Model trained in Section 3 using *group selected features*

2. Please provide potential explanations for the differences observed in models' performances across between section 1 and 4. (2 marks)

#### 4.2.3 - Answers
1.  Comparison of PCA Model with Other Models
- Section 1 (All Features): 95% accuracy, high performance across all classes, but might suffer from noise or overfitting due to redundant features.
- Section 2 (Selected Features): Likely similar performance to Section 1 but slightly reduced due to feature selection.
- Section 3 (Group-Selected Features): Similar to Section 2, possibly a slight dip in performance depending on feature grouping.
- Section 4 (PCA): 92% accuracy, performs slightly worse than Section 1 due to dimensionality reduction, but generalizes well by removing noise.
---
2. Explanations for Performance Differences Between Section 1 and Section 4

- PCA reduces dimensions, which may lose some useful features, causing a slight drop in accuracy.
- PCA focuses on the most important components, which may improve generalization but lose subtle details that the full feature set retains.

____

## Section 5 - Classification Using *Pre Selected Features* Derived From Inertial Sensor Data (10 Marks)

___

Now, a second classifer is to be trained and tuned using grid search. The **selected features from section 2** must be used. You may choose from one of the algorithms discussed that class listed below.
1. Random Forest
1. K-Nearest Neighbors
1. Gradient Boosting Machines
1. Neural Networks

____

### 5.1 Grid Search with Cross Validation (6 Marks)

___

#### 5.1.1 Extract *Selected* features and target from *cv_set*

**Questions**
1. Please list the ML model that you have selected . And explain *why you have chosen* this model? (2 marks)

**Answer**
1. Selected Model:
- For this section, Random Forest is chosen.
- Reason: Random Forest is a robust ensemble learning model that can handle high-dimensional data, like the selected features from Section 2. It’s less prone to overfitting than some other models and provides feature importance, which is useful in understanding the importance of each feature. Additionally, it performs well on datasets with complex, non-linear relationships.

In [ ]:
# TODO - Load group selected features from cv_set to X

X = cv_set[["a_max", "a_min", "a_var", "ax_max", "ax_min"]]
X.head(2)

,a_max,a_min,a_var,ax_max,ax_min
0,0.024997,0.023871,3.334598e-08,0.001496,-0.001539
1,0.024829,0.024153,1.588712e-08,0.001865,-0.001752


In [ ]:
# TODO - Load motion_state column (target vector) into y

y = cv_set["motion_state"]

___

#### 5.1.2 - Configure and Perform Grid Search with Cross-Validation

Similar to the steps from **1.2.2** and perform model training and tuning using for the model selected. Perform the `train_test_split`, initiate a `Pipeline`, then define the *Parameters* for `GridSearchCV`. Finally, perfrom the grid search.

In [ ]:
# TODO - Perform train/test split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Perform any missing imports
from sklearn.ensemble import RandomForestClassifier
# TODO - Initiate the pipeline
# (1 mark)

pipe = Pipeline([
    ('clf', RandomForestClassifier())
])

In [ ]:
# TODO - Define the parameter grid for your model
# (3 marks)

param_grid = {
    'clf__n_estimators': [100, 200],
    'clf__max_depth': [None, 10, 20],
    'clf__min_samples_split': [2, 5],
    'clf__min_samples_leaf': [1, 2],
    'clf__bootstrap': [True, False]
}

In [ ]:
# TODO - Perform the grid search

grid = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy', return_train_score=True)
grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('clf', RandomForestClassifier())]),
             param_grid={'clf__bootstrap': [True, False],
                         'clf__max_depth': [None, 10, 20],
                         'clf__min_samples_leaf': [1, 2],
                         'clf__min_samples_split': [2, 5],
                         'clf__n_estimators': [100, 200]},
             return_train_score=True, scoring='accuracy')

___

#### 5.1.3 - Visualize Grid Search Results

Print the results from the grid search. The following outputs must be printed.
* Best parameter set
* Best cross-validation train score
* Best cross-validation test score
* Test set accuracy

In [ ]:
# TODO: Print the results from the grid search

print("Best params:\n{}\n".format(grid.best_params_))
print("Best cross-validation train score: {:.2f}".format(grid.best_score_))
print("Best cross-validation test score: {:.2f}".format(grid.cv_results_['mean_test_score'][grid.best_index_]))
print("Test-set score: {:.2f}".format(grid.score(X_test, y_test)))

Best params:
{'clf__bootstrap': True, 'clf__max_depth': 10, 'clf__min_samples_leaf': 1, 'clf__min_samples_split': 5, 'clf__n_estimators': 100}

Best cross-validation train score: 0.79
Best cross-validation test score: 0.79
Test-set score: 0.81


Display the confusion matrix and print the classification report for the trained model using the *test* split.

In [ ]:
# TODO - Display confusion matrix and print classification report

y_pred = grid.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[101   3   0]
 [ 12 115  10]
 [  0  34  32]]
              precision    recall  f1-score   support

        rest       0.89      0.97      0.93       104
    straight       0.76      0.84      0.80       137
        turn       0.76      0.48      0.59        66

    accuracy                           0.81       307
   macro avg       0.80      0.77      0.77       307
weighted avg       0.80      0.81      0.80       307



___

### 5.2 - Model Evaluation on Test Datasets 1 and 2 (4 Marks)

___

#### 5.2.1 - Extract selected features and targets from test datasets

The model trained in 5.1, is to be tested on the independent datasets, that is test_set1 and test_set2.
Seperate the testsets into features and target vectors *(X1, y1)* and *(X2,y2)*

In [ ]:
# TODO - Load the target vector from test_set1 into y1, and the selected features from test_set1 into X1

y1 = test_set1["motion_state"]
X1 = test_set1[["a_max", "a_min", "a_var", "ax_max", "ax_min"]]  # Example feature names from Section 2
X1.head(2)

,a_max,a_min,a_var,ax_max,ax_min
0,0.024886,0.024056,3.437614e-08,0.002506,-0.001191
1,0.024883,0.024084,3.613404e-08,0.002427,-0.001104


In [ ]:
# TODO - Load the target vector from test_set2 into y2, and the selected features from test_set2 into X2

y2 = test_set2["motion_state"]
X2 = test_set2[["a_max", "a_min", "a_var", "ax_max", "ax_min"]]  # Example feature names from Section 2
X2.head(2)

,a_max,a_min,a_var,ax_max,ax_min
0,0.049764,0.048406,6.614559e-08,0.002575,-0.002970
1,0.050763,0.047696,3.344639e-07,0.003880,-0.003824


___

#### 5.2.2 - Visualize Results

For each test dataset, print the test set accuracy score.

In [ ]:
# TODO - Print test score accuracies for test_set1 and test_set2

print("test_set1 score: {:.2f}".format(grid.score(X1, y1)))
print("test_set2 score: {:.2f}".format(grid.score(X2, y2)))

test_set1 score: 0.78
test_set2 score: 0.72


Display the confusion matrix and print the classification for test_set 1 *(X1, y1)*

In [ ]:
# TODO - Display confusion matrix and print classification report for (X1, y1)

y1_pred = grid.predict(X1)
print("test_set1 Confusion Matrix:\n", confusion_matrix(y1, y1_pred))
print("test_set1 Classification Report:\n", classification_report(y1, y1_pred,))

test_set1 Confusion Matrix:
 [[192  39   0]
 [ 12 370  29]
 [  0  96  45]]
test_set1 Classification Report:
               precision    recall  f1-score   support

        rest       0.94      0.83      0.88       231
    straight       0.73      0.90      0.81       411
        turn       0.61      0.32      0.42       141

    accuracy                           0.78       783
   macro avg       0.76      0.68      0.70       783
weighted avg       0.77      0.78      0.76       783



Display the confusion matrix and print the classification for test_set 2 *(X2, y2)*

In [ ]:
# TODO - Display confusion matrix and print classification report for (X2, y2)

y2_pred = grid.predict(X2)
print("test_set2 Confusion Matrix:\n", confusion_matrix(y2, y2_pred))
print("test_set2 Classification Report:\n", classification_report(y2, y2_pred, zero_division = 0))

test_set2 Confusion Matrix:
 [[  0  16   0]
 [  0 200   6]
 [  0  62  13]]
test_set2 Classification Report:
               precision    recall  f1-score   support

        rest       0.00      0.00      0.00        16
    straight       0.72      0.97      0.83       206
        turn       0.68      0.17      0.28        75

    accuracy                           0.72       297
   macro avg       0.47      0.38      0.37       297
weighted avg       0.67      0.72      0.64       297



___

#### 5.2.3 Question

1. For the model trainied in this section, compare its performance to the performance of SVC which was also trained with the same selected features (section 2). Which model fits better? Explain causes for any observed discrepancies? (4 marks)

#### 5.2.3 - Answer
1. The **SVC model** outperformed the **Random Forest** model overall when using the same selected features. SVC demonstrated better accuracy and consistency, particularly for complex classes like **turn**. Random Forest performed well with the **rest** class but struggled with **turn**, especially on challenging datasets like **test_set2**. The differences are due to SVC's ability to handle complex decision boundaries better and its sensitivity to hyperparameters, which provided a tighter fit for overlapping classes. Random Forest’s reliance on individual feature splits and possible issues with data imbalance affected its performance. Thus, SVC fits better due to its adaptability to nuanced data patterns.


____